In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install imblearn
%pip install matplotlib
%pip install seaborn


# Reading and Parsing JSON

In [ ]:
# Specify the input and output file names (adjust file paths if needed)
input_file = '../data/dataset0.json.gz'  
output_file = '../data/dataset0.json'    

# Unzip the file
def unzip_file(input,output):
    import gzip
    import shutil
    with gzip.open(input_file, 'rb') as f_in:
        with open(output_file, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    print("File unzipped successfully.")

unzip_file(input_file,output_file)

In [ ]:
input_file = '../data/dataset0.json' 

def flatten_json(input_file):
    import json 
    import pandas as pd
    
    data = []
    with open(input_file, 'r') as file:
        for line in file:
            # Parse each line as a JSON object
            data.append(json.loads(line))
    
    rows = []

    for entry in data:
        for transcript_id, positions in entry.items():
            for position, sequences in positions.items():
                for sequence, features in sequences.items():
                    for feature_set in features:
                        row = {
                            'transcript_id': transcript_id,
                            'transcript_position': position,
                            'sequence': sequence,
                            '-1_dwelling_time': feature_set[0],
                            '-1_standard_dev': feature_set[1],
                            '-1_mean_current': feature_set[2],
                            '0_dwelling_time': feature_set[3],
                            '0_standard_dev': feature_set[4],
                            '0_mean_current': feature_set[5],
                            '+1_dwelling_time': feature_set[6],
                            '+1_standard_dev': feature_set[7],
                            '+1_mean_current': feature_set[8],
                        }
                        rows.append(row)

    df = pd.DataFrame(rows)
    return df

# Flatten the data
flattened_data = flatten_json(input_file)

In [ ]:
print(flattened_data.head())
print(flattened_data.info())

In [ ]:
# Export as csv
path = '../data/dataset0.csv'
flattened_data.to_csv(path, index=False)

# Load labels and features, perform aggregation

## Read in labels and features

In [ ]:
import pandas as pd
labels = pd.read_csv('../data/data.info.labelled')

print(labels.info())  # To get summary information about the DataFrame
print(labels.head())  # Preview the first few rows of the data

In [ ]:
features = pd.read_csv('../data/dataset0.csv')

print(features.info()) 
print(features.head())  

## Aggregate and label features

In [ ]:
def aggregate_by_transcript_position(features):

    # Apply mean, std, min, max, and skew to the selected columns
    features_agg = features.groupby(['transcript_id', 'transcript_position', 'sequence'])\
        .agg({
            '-1_dwelling_time': ['mean', 'min', 'max'],
            '-1_standard_dev': ['mean'],
            '-1_mean_current': ['mean', 'min', 'max'],
            '0_dwelling_time': ['mean', 'min', 'max'],
            '0_standard_dev': ['mean'],
            '0_mean_current': ['mean', 'min', 'max'],
            '+1_dwelling_time': ['mean', 'min', 'max'],
            '+1_standard_dev': ['mean'],
            '+1_mean_current': ['mean', 'min', 'max']
        }).reset_index()
    
    # Rename the columns to something more readable
    features_agg.columns = ['_'.join(col).strip() if col[1] else col[0] for col in features_agg.columns]

    

    return features_agg

In [ ]:
def one_hot_encode_sequence(features_agg, column='sequence'):
    # Step 1: Split each sequence into individual characters
    features_split = features_agg[column].apply(lambda x: pd.Series(list(x)))
    
    # Step 2: Remove the middle letter (always the 4th character, index 3 in 0-based index)
    features_split = features_split.drop(columns=[3])  # Drop the middle letter (index 3)
    
    # Step 3: One-hot encode the remaining letters
    # `pd.get_dummies` will automatically one-hot encode each position
    seq = pd.get_dummies(
        features_split,
        prefix=['pos1', 'pos2', 'pos3', 'pos5', 'pos6', 'pos7'],
        columns=[0, 1, 2, 4, 5, 6],
        dtype=int)

    expected_columns = [
        'pos1_A', 'pos1_T', 'pos1_C', 'pos1_G',
        'pos2_A', 'pos2_T', 'pos2_C', 'pos2_G',
        'pos3_A', 'pos3_T', 'pos3_C', 'pos3_G',
        'pos5_A', 'pos5_T', 'pos5_C', 'pos5_G',
        'pos6_A', 'pos6_T', 'pos6_C', 'pos6_G',
        'pos7_A', 'pos7_T', 'pos7_C', 'pos7_G'
    ]
    seq = seq.reindex(columns=expected_columns, fill_value=0)
    
    features_agg_with_seq = pd.concat([features_agg, seq], axis=1)
    features_agg_with_seq = features_agg_with_seq.drop(columns=[column])
    
    return features_agg_with_seq


In [ ]:
features_agg = aggregate_by_transcript_position(features)
features_agg_with_seq = one_hot_encode_sequence(features_agg)
print(features_agg_with_seq.info())
print(features_agg_with_seq.head())

In [ ]:
def add_gene_and_label(features, labels):
    """
    Adds gene_id and label to features dataframe
    
    Inputs:
    - features: pd.DataFrame
      Dataframe with selected features after feature engineering. Dataframe must contain transcript_id and transcript_position
    - labels: pd.DataFrame
      Dataframe with gene_id, transcript_id, transcript_position, and label.

    Output:
    - pd.DataFrame
      Updated features dataframe with added columns: gene_id and label from labels.
    """
    features_labelled = pd.merge(features, labels, on=['transcript_id', 'transcript_position'], how='inner')
    
    return features_labelled


In [ ]:
features_labelled = add_gene_and_label(features_agg, labels)
print(features_labelled.info())

In [ ]:
# features_labelled.to_csv('../data/features_labelled.csv',index=False)

# Train Test Split

In [ ]:
import pandas as pd
features_labelled = pd.read_csv('../data/features_labelled.csv')

In [ ]:
def train_test_split_by_gene_id(features_labelled, features_columns):
    """
    Performs train test split based on gene_id. Returns X_train and X_test based on feature_columns
    
    Inputs:
    - features_labelled: pd.DataFrame
      Updated features dataframe with added columns: gene_id and label from labels.
      
    Output:
    - X_train: pd.DataFrame
    - X_test: pd.DataFrame
    - y_train: pd.DataFrame
    - y_test: pd.DataFrame
    """

    from sklearn.model_selection import train_test_split
    df = features_labelled

    # Get unique genes
    unique_genes = df['gene_id'].unique()
    
    # Perform the train-test split on genes
    genes_train, genes_test = train_test_split(unique_genes, test_size=0.2, random_state=42)
    
    # Split the dataset based on the gene split
    train_data = df[df['gene_id'].isin(genes_train)]
    test_data = df[df['gene_id'].isin(genes_test)]
    
    # Create the feature and target variables for training and testing
    id_train = train_data[['transcript_id','transcript_position']]
    X_train = train_data[features_columns]
    y_train = train_data['label']
    id_test = test_data[['transcript_id','transcript_position']]
    X_test = test_data[features_columns]
    y_test = test_data['label']
    
    # Output the shapes to verify the split
    print(f"Training Features Shape: {X_train.shape}")
    print(f"Test Features Shape: {X_test.shape}")
    print(f"Training Labels Shape: {y_train.shape}")
    print(f"Test Labels Shape: {y_test.shape}")
    return (X_train, X_test, y_train, y_test, id_train, id_test)

In [ ]:
features_columns = [
        '-1_dwelling_time_mean', '-1_dwelling_time_min', '-1_dwelling_time_max',
        '-1_standard_dev_mean', 
        '-1_mean_current_mean', '-1_mean_current_min', '-1_mean_current_max',
        '0_dwelling_time_mean', '0_dwelling_time_min', '0_dwelling_time_max',
        '0_standard_dev_mean', 
        '0_mean_current_mean', '0_mean_current_min', '0_mean_current_max',
        '+1_dwelling_time_mean', '+1_dwelling_time_min', '+1_dwelling_time_max',
        '+1_standard_dev_mean', 
        '+1_mean_current_mean', '+1_mean_current_min', '+1_mean_current_max',
        'pos1_A', 'pos1_T', 'pos1_C', 'pos1_G',
        'pos2_A', 'pos2_T', 'pos2_C', 'pos2_G',
        'pos3_A', 'pos3_T', 'pos3_C', 'pos3_G',
        'pos5_A', 'pos5_T', 'pos5_C', 'pos5_G',
        'pos6_A', 'pos6_T', 'pos6_C', 'pos6_G',
        'pos7_A', 'pos7_T', 'pos7_C', 'pos7_G'
]
features_columns_means = [
    '-1_dwelling_time_mean',
    '-1_standard_dev_mean', 
    '-1_mean_current_mean',
    '0_dwelling_time_mean',
    '0_standard_dev_mean', 
    '0_mean_current_mean', 
    '+1_dwelling_time_mean',
    '+1_standard_dev_mean', 
    '+1_mean_current_mean'
]


In [ ]:
X_train_means_only, X_test_means_only, y_train, y_test, id_train, id_test = train_test_split_by_gene_id(features_labelled, features_columns_means)
X_train, X_test, y_train, y_test, id_train, id_test = train_test_split_by_gene_id(features_labelled, features_columns)

# Balancing Data

In [ ]:
def balance_train_data(X_train,y_train):
    """
    Performs SMOTE on train data, oversampling positive class, to account for imbalanced dataset
    
    Inputs:
    - X_train: pd.DataFrame
    - Y_train: pd.DataFrame
      
    Output:
    - X_train_resampled: pd.DataFrame
    - y_train_resampled: pd.DataFrame with balanced classes, ie the same number of 0s and 1s
    """

    from imblearn.over_sampling import SMOTE
    print(f'Label distribution before resampling:')
    print(pd.Series(y_train).value_counts())
    
    smote = SMOTE(k_neighbors=5, random_state=42) 
    X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)
    print(f'Label distribution after resampling:')
    print(pd.Series(y_train_resampled).value_counts())
    
    return X_train_resampled, y_train_resampled

# Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import confusion_matrix,accuracy_score,balanced_accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, precision_recall_curve, auc
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Predict and evaluate functions

In [ ]:
# Predict the output of xtest using the trained models
def predict(classifier, id, X):
    y_pred = classifier.predict(X)
    y_prob = classifier.predict_proba(X)[:, 1] #Gives probability estimates for y=1

    y_out = pd.DataFrame({
        'prediction': y_pred,                  
        'probability': y_prob          
    })

    result = pd.concat((id.reset_index(drop=True), y_out.reset_index(drop=True)), axis=1)
    # print(result.head())
    return result

In [ ]:
# Model evaluation
def evaluate(y_test, predict_df):
    y_pred = predict_df['prediction']
    y_prob = predict_df['probability']
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test,  y_pred)
    balancedaccuracy = balanced_accuracy_score(y_test,  y_pred)
    f1score = f1_score(y_test,  y_pred) #F1 is a good scoring metric for imbalanced data when more attention is needed on the positives
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_prob) 
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, y_prob) # Computes ROC AUC score using probabilities of positive class
    pr_auc = auc(recall_vals
                 , precision_vals)

    print(f"Accuracy = {round(accuracy, ndigits=3)}")
    print(f"Balanced Accuracy = {round(balancedaccuracy, ndigits=3)}")
    print(f"f1 score = {round(f1score, ndigits=3)}")
    print(f"Precision = {round(precision, ndigits=3)}")
    print(f"Recall = {round(recall, ndigits=3)}")
    print(f"ROC AUC = {round(roc_auc, ndigits=3)}")
    print(f"PR AUC = {round(pr_auc, ndigits=3)}")

    # Plot confusion matrix
    plt.figure(figsize=(2,2))
    sns.heatmap(cm, annot=True, fmt=".3f", linewidths=.5, square = True, cmap = 'Blues_r')
    plt.ylabel('Actual label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.figure(figsize=(5,5))  # Increase the figure size to avoid overlap
    plt.show()

    # Plot ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_prob)
    plt.plot(fpr, tpr, label='ROC curve (AUC = %0.2f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.show()

    # Plot Precision-Recall curve
    plt.plot(recall_vals, precision_vals, label='PR curve (AUC = %0.2f)' % pr_auc)
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Precision-Recall Curve')
    plt.show()

## Baseline Model - Logistic Regression with Means Only

In [ ]:
def LR(X_train,y_train,balance):
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)
    classifier = LogisticRegression(max_iter=1000, verbose = 1,  random_state = 123, class_weight = 'balanced')
    classifier.fit(X_train, y_train)
    return classifier

In [ ]:
LR_classifier_means = LR(X_train_means_only, y_train, False)
result = predict(LR_classifier_means, id_test, X_test_means_only)
evaluate(y_test, result)

## Logistic Regression

In [ ]:
# Logistic Regression Model
def LR(X_train,y_train, balance):
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)
    # remove low variance features
    # Define the pipeline with variance threshold selector and logistic regression classifier
    model = Pipeline([
        ('selector', VarianceThreshold(threshold=0.01)),
        ('classifier', LogisticRegression(max_iter=1000, verbose=1, random_state=123, class_weight='balanced'))
    ])
    # Train the model on X_train and y_train
    model.fit(X_train, y_train)
    return model

In [ ]:
# ML workflow for Logistic Regression - without SMOTE
LR_classifier = LR(X_train, y_train, False)
result = predict(LR_classifier, id_test, X_test)
evaluate(y_test, result)

In [ ]:
# ML workflow for Logistic Regression - with SMOTE
LR_classifier_balanced = LR(X_train, y_train,True)
result = predict(LR_classifier, id_test, X_test)
evaluate(y_test, result)


In [ ]:
selection_mask= LR_classifier_balanced.named_steps['selector'].get_support()

# Create a DataFrame with feature names and selection status
feature_selection_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Selected': selection_mask
})

count_df = pd.Series(LR_classifier.named_steps['selector'].get_support()).value_counts().reset_index(name='Count').rename(columns={'index': 'Selected'})

print(feature_selection_df)
print(count_df)

## Random Forest

In [ ]:
def RF(X_train, y_train):
    classifier = RandomForestClassifier(n_estimators=100, random_state=123, class_weight='balanced', verbose=1)
    classifier.fit(X_train, y_train)
    return classifier

In [ ]:
RF_classifier = RF(X_train, y_train)
result = predict(RF_classifier, id_test_clean, X_test_clean)
evaluate(y_test_clean, result)

In [ ]:
def RF_tuned_intermediate(X_train, y_train, balance, param_dist):
    '''
    RF_tuned_intermediate uses RandomizedSearch to reduce search space
    '''
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
    from sklearn.model_selection import RandomizedSearchCV
    import numpy as np
    
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)

    # Define the pipeline with variance threshold selector and random forest classifier
    model = RandomForestClassifier(random_state=123, class_weight='balanced')

    def combined_auc_score(y_true, y_pred_proba):
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        pr_auc = average_precision_score(y_true, y_pred_proba)
    
        # Combined score
        combined_score = roc_auc + pr_auc
        return combined_score

    # Create a custom scorer using make_scorer
    custom_scorer = make_scorer(combined_auc_score, needs_proba=True)

    # Set up RandomizedSearchCV
    randomized_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist,
                                           n_iter=30, scoring=custom_scorer, cv=3, n_jobs=-1, verbose=1, random_state=123)
    
    # Train the model on X_train and y_train using RandomizedSearchCV
    randomized_search.fit(X_train, y_train)

    
    return randomized_search.best_estimator_, randomized_search.best_params_

def RF_tuned(X_train, y_train, balance, param_grid):
    '''
    RF_tuned uses GridSearch on an exhaustive list of parameters, determined after GridSearch
    '''
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import make_scorer, roc_auc_score, average_precision_score
    from sklearn.model_selection import GridSearchCV
    
     # Balance the data if balance is True
    if balance:
        X_train, y_train = balance_train_data(X_train, y_train)

    # Define the random forest model with class_weight set to 'balanced'
    model = RandomForestClassifier(random_state=123, class_weight='balanced')

    # Custom scoring function combining ROC AUC and PR AUC
    def combined_auc_score(y_true, y_pred_proba):
        roc_auc = roc_auc_score(y_true, y_pred_proba)
        pr_auc = average_precision_score(y_true, y_pred_proba)
    
        # Combined score
        combined_score = roc_auc + pr_auc
        return combined_score

    # Create a custom scorer using make_scorer
    custom_scorer = make_scorer(combined_auc_score, needs_proba=True)

    # Set up GridSearchCV with the parameter grid
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid,
                               scoring=custom_scorer, cv=5, n_jobs=-1, verbose=1)
    
    # Train the model on X_train and y_train using GridSearchCV
    grid_search.fit(X_train, y_train)

    return grid_search.best_estimator_, grid_search.best_params_

In [ ]:
param_dist_search_1 = {
            'n_estimators': [100, 200, 300, 400, 500],  
            'max_depth': [None, 10, 20, 30, 40, 50],  
            'min_samples_split': [2,3,4,5], 
            'min_samples_leaf': [1,2,3,4,5],  
            'max_features': ['sqrt', 'log2'],  
            'bootstrap': [True, False]  
        }    
RF_search_1_classifier, RF_search_1_params = RF_tuned_intermediate(X_train, y_train, False, param_dist_search_1)
print(f'Tuning step 1 outputs: {RF_search_1_params}')

In [ ]:
param_dist_search_2 = {
            'n_estimators': [400, 405, 410, 415, 420, 425],  
            'max_depth': [None, 40, 41, 42, 43, 44, 45],
            'min_samples_split': [3, 4, 5], 
            'min_samples_leaf': [1, 2, 3],  
            'max_features': ['sqrt'],  
            'bootstrap': [False]  
        }    
RF_search_2_classifier, RF_search_2_params = RF_tuned_intermediate(X_train, y_train, False, param_dist_search_2)
print(f'Tuning step 2 outputs: {RF_search_2_params}')

In [ ]:
# Hyperparameter grid for exhaustive search based on result from step 1
param_grid_final = {
    'n_estimators': [225, 250, 275], 
    'max_depth': [None],  
    'min_samples_split': [3, 4], 
    'min_samples_leaf': [1, 2], 
    'max_features': ['sqrt'],  
    'bootstrap': [False] 
}

RF_tuned, RF_tuned_params = RF_tuned(X_train, y_train, param_grid_final, False)
print(RF_tuned_params)
result = predict(RF_tuned_classifier, id_test, X_test)
evaluate(y_test, result)

In [ ]:
def plot_rfe_graph(X_train, y_train, tuned_model, min_features=10, step=10):

    import numpy as np
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.feature_selection import RFE
    from sklearn.metrics import roc_auc_score
    import matplotlib.pyplot as plt
    
    # Ensure the tuned model is a Random Forest
    if not isinstance(tuned_model, RandomForestClassifier):
        raise ValueError("Tuned model must be a Random Forest classifier.")
    
    # Use RFE with the tuned Random Forest as the estimator
    rfe = RFE(estimator=tuned_model, n_features_to_select=min_features, step=step)
    
    # Fit RFE once and get ranking
    rfe.fit(X_train, y_train)
    rfe_ranking = rfe.ranking_
    
    # Evaluate performance for each number of features in decreasing order
    feature_counts = np.arange(X_train.shape[1], min_features - 1, -step)
    roc_auc_scores = []
    
    for n_features in feature_counts:
        # Adjust the number of features to select
        rfe.n_features_to_select = n_features
        X_rfe = X_train.loc[:, rfe.support_]
        
        # Fit the tuned model directly on the reduced feature set
        tuned_model.fit(X_rfe, y_train)
        y_pred_proba = tuned_model.predict_proba(X_rfe)[:, 1]
        
        # Calculate ROC AUC and store it
        roc_auc = roc_auc_score(y_train, y_pred_proba)
        roc_auc_scores.append(roc_auc)
    
    # Plot ROC AUC scores against the number of features
    plt.figure(figsize=(10, 6))
    plt.plot(feature_counts, roc_auc_scores, marker='o')
    plt.title('ROC AUC Score vs. Number of Features (RFE)')
    plt.xlabel('Number of Features Selected')
    plt.ylabel('ROC AUC Score')
    plt.xticks(feature_counts)
    plt.grid()
    plt.show()
    
    # Create a DataFrame to show feature rankings and names
    feature_names = X_train.columns
    feature_ranking_df = pd.DataFrame({'Feature': feature_names, 'Ranking': rfe_ranking})
    selected_features = feature_ranking_df[feature_ranking_df['Ranking'] == 1]['Feature']
    
    return selected_features.tolist()

In [ ]:
selected_features = plot_rfe_graph(X_train, y_train, RF_tuned_classifier)
print(selected_features)

In [ ]:
from joblib import dump

# After defining RF_classifier
dump(RF_classifier, 'rf_classifier.joblib')  # Save the model


# Predictions for dataset0

In [ ]:
X_test_0 = features_agg[features_columns]
id_test_0 = features_agg[['transcript_id', 'transcript_position']]

In [ ]:
result = predict(LR_classifier, id_test_0, X_test_0)

result = result.sort_values(by='probability', ascending=False)
result = result.rename(columns={'probability': 'score'})
result = result[['transcript_id', 'transcript_position', 'score']]


In [ ]:
# Export results to CSV
output_csv_path = '../data/geneiuses_dataset0_1.csv'
result.to_csv(output_csv_path, index=False)

print(f"Results exported to {output_csv_path}")


# Predictions for dataset1

In [ ]:
import gzip
import shutil

# Specify the input and output file names (adjust file paths if needed)
input_file_1 = '../data/dataset1.json.gz'  
output_file_1 = '../data/dataset1.json'    

# Unzip the file
with gzip.open(input_file_1, 'rb') as f_in:
    with open(output_file_1, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File unzipped successfully.")

In [ ]:
import pandas as pd
import json

input_file_1 = '../data/dataset1.json' 

data_1 = []
with open(input_file_1, 'r') as file:
    for line in file:
        # Parse each line as a JSON object
        data_1.append(json.loads(line))

# Flatten the data
flattened_data_1 = flatten_json(data_1)

# Convert to a DataFrame
df_1 = pd.DataFrame(flattened_data_1)

In [ ]:
# Export as csv
path = '../data/dataset1.csv'
df_1.to_csv(path, index=False)

In [ ]:
features_1 = pd.read_csv('../data/dataset1.csv')
features_agg_1 = aggregate_by_transcript_position(features_1)
features_columns_1 = [
        '-1_dwelling_time', '-1_standard_dev', '-1_mean_current',
        '0_dwelling_time', '0_standard_dev', '0_mean_current',
        '+1_dwelling_time', '+1_standard_dev', '+1_mean_current'
    ]

X_test_1 = features_agg_1[features_columns_1]
id_test_1 = features_agg_1[['transcript_id', 'transcript_position']]

In [ ]:
result_1 = predict(LR_classifier, id_test_1, X_test_1)
result_1 = result_1.sort_values(by='probability', ascending=False)
result_1 = result_1.rename(columns={'probability': 'score'})
result_1 = result_1[['transcript_id', 'transcript_position', 'score']]


In [ ]:
# Export results to CSV
output_csv_path = '../data/geneiuses_dataset1_1.csv'
result_1.to_csv(output_csv_path, index=False)

print(f"Results exported to {output_csv_path}")


# Predictions for dataset2

In [ ]:
import gzip
import shutil

# Specify the input and output file names (adjust file paths if needed)
input_file_2 = '../data/dataset2.json.gz'  
output_file_2 = '../data/dataset2.json'    

# Unzip the file
with gzip.open(input_file_2, 'rb') as f_in:
    with open(output_file_2, 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("File unzipped successfully.")

In [ ]:
import pandas as pd
import json

input_file_2 = '../data/dataset2.json' 

data_2 = []
with open(input_file_2, 'r') as file:
    for line in file:
        # Parse each line as a JSON object
        data_2.append(json.loads(line))

# Flatten the data
flattened_data_2 = flatten_json(data_2)

# Convert to a DataFrame
df_2 = pd.DataFrame(flattened_data_2)

In [ ]:
# Export as csv
path = '../data/dataset2.csv'
df_2.to_csv(path, index=False)

In [ ]:
features_2 = pd.read_csv('../data/dataset2.csv')
features_agg_2 = aggregate_by_transcript_position(features_2)
features_columns_2 = [
        '-1_dwelling_time', '-1_standard_dev', '-1_mean_current',
        '0_dwelling_time', '0_standard_dev', '0_mean_current',
        '+1_dwelling_time', '+1_standard_dev', '+1_mean_current'
    ]

X_test_2 = features_agg_2[features_columns_2]
id_test_2 = features_agg_2[['transcript_id', 'transcript_position']]


In [ ]:
# ML workflow for Logistic Regression
result_2 = predict(LR_classifier, id_test_2, X_test_2)

In [ ]:
result_2 = result_2.sort_values(by='probability', ascending=False)
result_2 = result_2.rename(columns={'probability': 'score'})
result_2 = result_2[['transcript_id', 'transcript_position', 'score']]


In [ ]:
# Export results to CSV
output_csv_path = '../data/geneiuses_dataset2_1.csv'
result_2.to_csv(output_csv_path, index=False)

print(f"Results exported to {output_csv_path}")
